In [1]:
# Write a program that will iteratively update and
# predict based on the location measurements 
# and inferred motions shown below. 

def update(mean1, var1, mean2, var2):
    new_mean = float(var2 * mean1 + var1 * mean2) / (var1 + var2)
    new_var = 1./(1./var1 + 1./var2)
    return [new_mean, new_var]

def predict(mean1, var1, mean2, var2):
    new_mean = mean1 + mean2
    new_var = var1 + var2
    return [new_mean, new_var]

measurements = [5., 6., 7., 9., 10.,11.,12.,13.]
motion = [1., 1., 2., 1., 1.,1.,1.,1.]
measurement_sig = 0.4
motion_sig = 2.
mu = 200.
sig = 10.

#Please print out ONLY the final values of the mean
#and the variance in a list [mu, sig]. 

# Insert code here
print(measurements)
for i in range(len(motion)):
    [mu, sig] = update(mu, sig, measurements[i], measurement_sig)
    [mu, sig] = predict(mu,sig, motion[i], motion_sig)
print ("after motion",[mu, sig])

[5.0, 6.0, 7.0, 9.0, 10.0, 11.0, 12.0, 13.0]
after motion [14.000010387401607, 2.3416407864999575]


In [80]:
print ('hello')

hello


In [39]:
# Write a function 'kalman_filter' that implements a multi-
# dimensional Kalman Filter for the example given

from math import *

class matrix:
    
    # implements basic operations of a matrix class
    
    def __init__(self, value):
        self.value = value
        self.dimx = len(value)
        self.dimy = len(value[0])
        if value == [[]]:
            self.dimx = 0
    
    def zero(self, dimx, dimy):
        # check if valid dimensions
        if dimx < 1 or dimy < 1:
              print( "Invalid size of matrix")
        else:
            self.dimx = dimx
            self.dimy = dimy
            self.value = [[0 for row in range(dimy)] for col in range(dimx)]
    
    def identity(self, dim):
        # check if valid dimension
        if dim < 1:
              print(  "Invalid size of matrix")
        else:
            self.dimx = dim
            self.dimy = dim
            self.value = [[0 for row in range(dim)] for col in range(dim)]
            for i in range(dim):
                self.value[i][i] = 1
    
    def show(self):
        for i in range(self.dimx):
            print (self.value[i])
        print (' ')
    
    def __add__(self, other):
        # check if correct dimensions
        if self.dimx != other.dimx or self.dimy != other.dimy:
              print(  "Matrices must be of equal dimensions to add")
        else:
            # add if correct dimensions
            res = matrix([[]])
            res.zero(self.dimx, self.dimy)
            for i in range(self.dimx):
                for j in range(self.dimy):
                    res.value[i][j] = self.value[i][j] + other.value[i][j]
            return res
    
    def __sub__(self, other):
        # check if correct dimensions
        if self.dimx != other.dimx or self.dimy != other.dimy:
              print(  "Matrices must be of equal dimensions to subtract")
        else:
            # subtract if correct dimensions
            res = matrix([[]])
            res.zero(self.dimx, self.dimy)
            for i in range(self.dimx):
                for j in range(self.dimy):
                    res.value[i][j] = self.value[i][j] - other.value[i][j]
            return res
    
    def __mul__(self, other):
        # check if correct dimensions
        if self.dimy != other.dimx:
              print(  "Matrices must be m*n and n*p to multiply")
        else:
            # subtract if correct dimensions
            res = matrix([[]])
            res.zero(self.dimx, other.dimy)
            for i in range(self.dimx):
                for j in range(other.dimy):
                    for k in range(self.dimy):
                        res.value[i][j] += self.value[i][k] * other.value[k][j]
            return res
    
    def transpose(self):
        # compute transpose
        res = matrix([[]])
        res.zero(self.dimy, self.dimx)
        for i in range(self.dimx):
            for j in range(self.dimy):
                res.value[j][i] = self.value[i][j]
        return res
    
    # Thanks to Ernesto P. Adorio for use of Cholesky and CholeskyInverse functions
    
    def Cholesky(self, ztol=1.0e-5):
        # Computes the upper triangular Cholesky factorization of
        # a positive definite matrix.
        res = matrix([[]])
        res.zero(self.dimx, self.dimx)
        
        for i in range(self.dimx):
            S = sum([(res.value[k][i])**2 for k in range(i)])
            d = self.value[i][i] - S
            if abs(d) < ztol:
                res.value[i][i] = 0.0
            else:
                if d < 0.0:
                    print( "Matrix not positive-definite")
                res.value[i][i] = sqrt(d)
            for j in range(i+1, self.dimx):
                S = sum([res.value[k][i] * res.value[k][j] for k in range(self.dimx)])
                if abs(S) < ztol:
                    S = 0.0
                res.value[i][j] = (self.value[i][j] - S)/res.value[i][i]
        return res
    
    def CholeskyInverse(self):
        # Computes inverse of matrix given its Cholesky upper Triangular
        # decomposition of matrix.
        res = matrix([[]])
        res.zero(self.dimx, self.dimx)
        
        # Backward step for inverse.
        for j in reversed(range(self.dimx)):
            tjj = self.value[j][j]
            S = sum([self.value[j][k]*res.value[j][k] for k in range(j+1, self.dimx)])
            res.value[j][j] = 1.0/tjj**2 - S/tjj
            for i in reversed(range(j)):
                res.value[j][i] = res.value[i][j] = -sum([self.value[i][k]*res.value[k][j] for k in range(i+1, self.dimx)])/self.value[i][i]
        return res
    
    def inverse(self):
        aux = self.Cholesky()
        res = aux.CholeskyInverse()
        return res
    
    def __repr__(self):
        return repr(self.value)


########################################

# Implement the filter function below

def kalman_filter(x, P):
    
    for n in range(len(measurements)):
        # measurement updat
        Z=matrix([[measurements[n]]])
        y= Z-(H*x)
        S = H*P*H.transpose()+R
        K=P*H.transpose()*S.inverse()
        x =x+ K*y
        P = (I- K*H)*P
       

        # prediction
        x = F*x+u
        P = F*P*F.transpose()
    print (x)
    print(P)
    

############################################
### use the code below to test your filter!
############################################



measurements = [1, 2, 3]
dt=1.

x = matrix([[0.], [0.]]) # initial state (location and velocity)
P = matrix([[1000., 0.], [0., 1000.]]) # initial uncertainty
u = matrix([[0.], [0.]]) # external motion
F = matrix([[1., dt], [0, dt]]) # next state function
H = matrix([[1., 0.]]) # measurement function
R = matrix([[1.]]) # measurement uncertainty
I = matrix([[1., 0.], [0., 1.]]) # identity matrix


kalman_filter(x, P)
# output should be:
# x: [[3.9996664447958645], [0.9999998335552873]]
# P: [[2.3318904241194827, 0.9991676099921091], [0.9991676099921067, 0.49950058263974184]]


[[3.9996664447958645], [0.9999998335552873]]
[[2.3318904241194827, 0.9991676099921091], [0.9991676099921067, 0.49950058263974184]]


In [40]:
# Fill in the matrices P, F, H, R and I at the bottom
#
# This question requires NO CODING, just fill in the 
# matrices where indicated. Please do not delete or modify
# any provided code OR comments. Good luck!

from math import *

class matrix:
    
    # implements basic operations of a matrix class
    
    def __init__(self, value):
        self.value = value
        self.dimx = len(value)
        self.dimy = len(value[0])
        if value == [[]]:
            self.dimx = 0
    
    def zero(self, dimx, dimy):
        # check if valid dimensions
        if dimx < 1 or dimy < 1:
            print( "Invalid size of matrix")
        else:
            self.dimx = dimx
            self.dimy = dimy
            self.value = [[0 for row in range(dimy)] for col in range(dimx)]
    
    def identity(self, dim):
        # check if valid dimension
        if dim < 1:
             print( "Invalid size of matrix")
        else:
            self.dimx = dim
            self.dimy = dim
            self.value = [[0 for row in range(dim)] for col in range(dim)]
            for i in range(dim):
                self.value[i][i] = 1
    
    def show(self):
        for i in range(self.dimx):
            print (self.value[i])
        print (' ')
    
    def __add__(self, other):
        # check if correct dimensions
        if self.dimx != other.dimx or self.dimy != other.dimy:
            print( "Matrices must be of equal dimensions to add")
        else:
            # add if correct dimensions
            res = matrix([[]])
            res.zero(self.dimx, self.dimy)
            for i in range(self.dimx):
                for j in range(self.dimy):
                    res.value[i][j] = self.value[i][j] + other.value[i][j]
            return res
    
    def __sub__(self, other):
        # check if correct dimensions
        if self.dimx != other.dimx or self.dimy != other.dimy:
            print( "Matrices must be of equal dimensions to subtract")
        else:
            # subtract if correct dimensions
            res = matrix([[]])
            res.zero(self.dimx, self.dimy)
            for i in range(self.dimx):
                for j in range(self.dimy):
                    res.value[i][j] = self.value[i][j] - other.value[i][j]
            return res
    
    def __mul__(self, other):
        # check if correct dimensions
        if self.dimy != other.dimx:
             print( "Matrices must be m*n and n*p to multiply")
        else:
            # subtract if correct dimensions
            res = matrix([[]])
            res.zero(self.dimx, other.dimy)
            for i in range(self.dimx):
                for j in range(other.dimy):
                    for k in range(self.dimy):
                        res.value[i][j] += self.value[i][k] * other.value[k][j]
            return res
    
    def transpose(self):
        # compute transpose
        res = matrix([[]])
        res.zero(self.dimy, self.dimx)
        for i in range(self.dimx):
            for j in range(self.dimy):
                res.value[j][i] = self.value[i][j]
        return res
    
    # Thanks to Ernesto P. Adorio for use of Cholesky and CholeskyInverse functions
    
    def Cholesky(self, ztol=1.0e-5):
        # Computes the upper triangular Cholesky factorization of
        # a positive definite matrix.
        res = matrix([[]])
        res.zero(self.dimx, self.dimx)
        
        for i in range(self.dimx):
            S = sum([(res.value[k][i])**2 for k in range(i)])
            d = self.value[i][i] - S
            if abs(d) < ztol:
                res.value[i][i] = 0.0
            else:
                if d < 0.0:
                     print( "Matrix not positive-definite")
                res.value[i][i] = sqrt(d)
            for j in range(i+1, self.dimx):
                S = sum([res.value[k][i] * res.value[k][j] for k in range(self.dimx)])
                if abs(S) < ztol:
                    S = 0.0
                res.value[i][j] = (self.value[i][j] - S)/res.value[i][i]
        return res
    
    def CholeskyInverse(self):
        # Computes inverse of matrix given its Cholesky upper Triangular
        # decomposition of matrix.
        res = matrix([[]])
        res.zero(self.dimx, self.dimx)
        
        # Backward step for inverse.
        for j in reversed(range(self.dimx)):
            tjj = self.value[j][j]
            S = sum([self.value[j][k]*res.value[j][k] for k in range(j+1, self.dimx)])
            res.value[j][j] = 1.0/tjj**2 - S/tjj
            for i in reversed(range(j)):
                res.value[j][i] = res.value[i][j] = -sum([self.value[i][k]*res.value[k][j] for k in range(i+1, self.dimx)])/self.value[i][i]
        return res
    
    def inverse(self):
        aux = self.Cholesky()
        res = aux.CholeskyInverse()
        return res
    
    def __repr__(self):
        return repr(self.value)


########################################

def filter(x, P):
    for n in range(len(measurements)):
        
        # prediction
        x = (F * x) + u
        P = F * P * F.transpose()
        
        # measurement update
        Z = matrix([measurements[n]])
        y = Z.transpose() - (H * x)
        S = H * P * H.transpose() + R
        K = P * H.transpose() * S.inverse()
        x = x + (K * y)
        P = (I - (K * H)) * P
    
    print ('x= ')
    x.show()
    print ('P= ')
    P.show()

########################################

print ("### 4-dimensional example ###")

measurements = [[5., 10.], [6., 8.], [7., 6.], [8., 4.], [9., 2.], [10., 0.]]
initial_xy = [4., 12.]

# measurements = [[1., 4.], [6., 0.], [11., -4.], [16., -8.]]
# initial_xy = [-4., 8.]

# measurements = [[1., 17.], [1., 15.], [1., 13.], [1., 11.]]
# initial_xy = [1., 19.]

dt = 0.1

x = matrix([[initial_xy[0]], [initial_xy[1]], [0.], [0.]]) # initial state (location and velocity)
u = matrix([[0.], [0.], [0.], [0.]]) # external motion

#### DO NOT MODIFY ANYTHING ABOVE HERE ####
#### fill this in, remember to use the matrix() function!: ####
P = matrix([[0., 0., 0., 0.], [0., 0., 0., 0.], [0., 0., 1000., 0.], [0., 0., 0., 1000.]]) # initial uncertainty: 0 for positions x and y, 1000 for the two velocities
F = matrix([[1., 0., dt, 0.],[0., 1., 0., dt],[0., 0., 1., 0.],[0., 0., 0., 1.]]) # next state function: generalize the 2d version to 4d
H = matrix([[1., 0., 0., 0.], [0., 1., 0., 0.]]) # measurement function: reflect the fact that we observe x and y but not the two velocities
R = matrix([[dt, 0.], [0., dt]]) # measurement uncertainty: use 2x2 matrix with 0.1 as main diagonal
I = matrix([[1., 0., 0., 0.], [0., 1., 0., 0.], [0., 0., 1., 0.], [0., 0., 0., 1.]]) # 4d identity matrix
###### DO NOT MODIFY ANYTHING HERE #######

filter(x, P)


### 4-dimensional example ###
x= 
[9.999340731787717]
[0.001318536424568617]
[9.998901219646193]
[-19.997802439292386]
 
P= 
[0.03955609273706198, 0.0, 0.06592682122843721, 0.0]
[0.0, 0.03955609273706198, 0.0, 0.06592682122843721]
[0.06592682122843718, 0.0, 0.10987803538073201, 0.0]
[0.0, 0.06592682122843718, 0.0, 0.10987803538073201]
 


In [395]:
#QUESTION NUMBER 2
#ANSWER IS RMSE = [[7.41826686 8.67626569 4.05611556 3.85419828]]
import pandas as pd
import numpy as np
from math import *

def computeRMSE(EstimateVector, trueVector):
    if len(EstimateVector) != len(trueVector) or len(EstimateVector) == 0:
        raise ValueError('computeRMSE() - Error - EstimateVector and trueVector not the same length.')

    rmse = np.matrix([0., 0., 0., 0.]).T
    for est, tru in zip(EstimateVector, trueVector):
        rmse += np.square(est - tru)

    rmse /= len(EstimateVector)
    return np.sqrt(rmse)

def computeRadarJacobian(Xvector):
    Px,Py,vx,vy = (Xvector[0][0,0],Xvector[1][0,0],Xvector[2][0,0],Xvector[3][0,0])
    
    px2_py2 = Px**2 + Py**2
    sqrt_px2_py2 = sqrt(px2_py2)
    px2_py2_sqrt_px2_py2 = (px2_py2 * sqrt_px2_py2)

    if px2_py2 < 0.0001:
        return np.matrix([[0, 0, 0, 0],
                         [0, 0, 0, 0],
                         [0, 0, 0, 0]])

    place11 = Px / sqrt_px2_py2
    place12 = Py / sqrt_px2_py2
    place21 = -Py / px2_py2
    place22 = Px / px2_py2
    place31 = Py * (vx * Py - vy * Px) / px2_py2_sqrt_px2_py2
    place32 = Px * (Px * vy - Py * vx) / px2_py2_sqrt_px2_py2

    return np.matrix([[place11, place12, 0, 0],
                      [place21, place22, 0, 0],
                      [place31, place32, place11, place12]])
def computeCovMatrix(deltaT, sigma_aX, sigma_aY):
    pass
def computeFmatrix(deltaT):
    mat = np.matrix([[1, 0, deltaT, 0],
           [0, 1, 0, deltaT],
           [0, 0, 1, 0],
           [0, 0, 0, 1]])
    return mat
def computeQmatrix(dt, noise_ax, noise_ay):
    delta2 = dt**2
    delta3 = dt**3
    delta4 = dt**4

    mat = np.matrix([[delta4 * noise_ax / 4, 0, delta3 * noise_ax / 2, 0],
                     [0, delta4 * noise_ay / 4, 0, delta3 * noise_ay / 2],
                     [delta3 * noise_ax / 2, 0, delta2 * noise_ax, 0],
                     [0, delta3 * noise_ay / 2, 0, delta2 * noise_ay]])
    return mat




data = pd.read_csv("obj_pose-laser-radar-synthetic-input.txt",                  
                header=None,                           
                sep='\t',                              
                names=['x'+str(x) for x in range(11)]) 
dt = 0.1
timestamp, prev_timestamp = 0, 0
Px, Py, vx, vy = 0, 0, 0, 0
x = None
P = np.matrix([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1000, 0], [0, 0, 0, 1000]])
HL = np.matrix([[1, 0, 0, 0], [0, 1, 0, 0]])
HR = np.matrix([[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]])
RL = np.matrix([[0.0225, 0], [0, 0.0225]])
RR = np.matrix([[0.09, 0, 0], [0, 0.0009, 0], [0, 0, 0.09]])
F = computeFmatrix(dt)
xI = np.eye(4)
z  = 0
estimations = []   
measurements = []  

only_lidar = True
is_initialized = False
for _, packet in data.iterrows(): 
    if packet[0] == 'L':
        z = np.matrix([[packet[1], packet[2]]]).T
        timestamp = packet[3]
        measurements.append(np.matrix([[packet[4]],
                                           [packet[5]],
                                           [packet[6]],
                                           [packet[7]]]))
        
            
    elif packet[0] == 'R' and not only_lidar:
        z = np.matrix([[packet[1], packet[2], packet[3]]]).T
        timestamp = packet[4]
        measurements.append(np.matrix([[packet[5]],
                                           [packet[6]],
                                           [packet[7]],
                                           [packet[8]]]))
        
    if not is_initialized:
        if packet[0] == 'L':
            Px, Py, vx, vy = packet[1], packet[2], 0, 0
        elif packet[0] == 'R' and not only_lidar:
            phi_measured = packet[1]
            rho_measured = packet[2]
            rhodot_measured = packet[3]
            Px, Py = (cos(phi_measured) * rho_measured, sin(phi_measured) * rho_measured)
            vx, vy = (rhodot_measured * cos(phi_measured) , rhodot_measured * sin(phi_measured))
        if abs(Px + Py) <= 0.0001:
            Px = 0.0001
            Py = 0.0001
        x = np.matrix([[Px,Py,vx,vy]]).T
        prev_timestamp = timestamp
        is_initialized = True
        estimations.append(x)
    else:
        dt = (timestamp - prev_timestamp) / 1000000.0
        prev_timestamp = timestamp
        F = computeFmatrix(dt)
        
        x = F * x
        P = (F * P * F.T)
        
        if packet[0] == 'L':
            y = z - HL * x

            PHLt = P * HL.T
            S = HL * PHLt + RL
            K = PHLt * S.I

            x += K * y
            P = (xI - K * HL) * P
            estimations.append(x)
        elif packet[0] == 'R' and not only_lidar:
            Px,Py,vx,vy = x[0][0,0], x[1][0,0], x[2][0,0], x[3][0,0]
            ro = sqrt(Px**2 + Py**2)

            phi = atan2(Py,Px)
            ro_dot = (Px*vx + Py*vy) / ro
            
            y = z - np.matrix([ro, phi, ro_dot]).T
            
            while (y[1] > pi): y[1] -= 2.*pi
            while (y[1] < -pi): y[1] += 2.*pi
            
            HR = computeRadarJacobian(x)
            
            S = HR * P * HR.T + RR
            K = P * HR.T * S.I
            x += K * y


            P = (xI - K * HR) * P
            estimations.append(x)
        
rmse = computeRMSE(estimations, measurements).flatten()
print("RMSE = {}".format(rmse))

RMSE = [[7.41826686 8.67626569 4.05611556 3.85419828]]


In [396]:
#QUESTION NUMBER 3
#ANSWER IS RMSE = [[0.12219136 0.09837984 0.58251275 0.45669849]]
import pandas as pd
import numpy as np
from math import *

def computeRMSE(EstimateVector, trueVector):
    if len(EstimateVector) != len(trueVector) or len(EstimateVector) == 0:
        raise ValueError('computeRMSE() - Error - EstimateVector and trueVector not the same length.')

    rmse = np.matrix([0., 0., 0., 0.]).T
    for est, tru in zip(EstimateVector, trueVector):
        rmse += np.square(est - tru)

    rmse /= len(EstimateVector)
    return np.sqrt(rmse)

def computeRadarJacobian(Xvector):
    Px,Py,vx,vy = (Xvector[0][0,0],Xvector[1][0,0],Xvector[2][0,0],Xvector[3][0,0])
    
    px2_py2 = Px**2 + Py**2
    sqrt_px2_py2 = sqrt(px2_py2)
    px2_py2_sqrt_px2_py2 = (px2_py2 * sqrt_px2_py2)

    if px2_py2 < 0.0001:
        return np.matrix([[0, 0, 0, 0],
                         [0, 0, 0, 0],
                         [0, 0, 0, 0]])

    place11 = Px / sqrt_px2_py2
    place12 = Py / sqrt_px2_py2
    place21 = -Py / px2_py2
    place22 = Px / px2_py2
    place31 = Py * (vx * Py - vy * Px) / px2_py2_sqrt_px2_py2
    place32 = Px * (Px * vy - Py * vx) / px2_py2_sqrt_px2_py2

    return np.matrix([[place11, place12, 0, 0],
                      [place21, place22, 0, 0],
                      [place31, place32, place11, place12]])
def computeCovMatrix(deltaT, sigma_aX, sigma_aY):
    pass
def computeFmatrix(deltaT):
    mat = np.matrix([[1, 0, deltaT, 0],
           [0, 1, 0, deltaT],
           [0, 0, 1, 0],
           [0, 0, 0, 1]])
    return mat
def computeQmatrix(dt, noise_ax, noise_ay):
    delta2 = dt**2
    delta3 = dt**3
    delta4 = dt**4

    mat = np.matrix([[delta4 * noise_ax / 4, 0, delta3 * noise_ax / 2, 0],
                     [0, delta4 * noise_ay / 4, 0, delta3 * noise_ay / 2],
                     [delta3 * noise_ax / 2, 0, delta2 * noise_ax, 0],
                     [0, delta3 * noise_ay / 2, 0, delta2 * noise_ay]])
    return mat




data = pd.read_csv("obj_pose-laser-radar-synthetic-input.txt",                  
                header=None,                           
                sep='\t',                              
                names=['x'+str(x) for x in range(11)]) 
dt = 0.1
timestamp, prev_timestamp = 0, 0
Px, Py, vx, vy = 0, 0, 0, 0
x = None
P = np.matrix([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1000, 0], [0, 0, 0, 1000]])
HL = np.matrix([[1, 0, 0, 0], [0, 1, 0, 0]])
HR = np.matrix([[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]])
RL = np.matrix([[0.0225, 0], [0, 0.0225]])
RR = np.matrix([[0.09, 0, 0], [0, 0.0009, 0], [0, 0, 0.09]])
F = computeFmatrix(dt)
Q = None
xI = np.eye(4)
z  = 0
estimations = []   
measurements = []  

only_lidar = True
is_initialized = False
for _, packet in data.iterrows(): 
    if packet[0] == 'L':
        z = np.matrix([[packet[1], packet[2]]]).T
        timestamp = packet[3]
        measurements.append(np.matrix([[packet[4]],
                                           [packet[5]],
                                           [packet[6]],
                                           [packet[7]]]))
        
            
    elif packet[0] == 'R' and not only_lidar:
        z = np.matrix([[packet[1], packet[2], packet[3]]]).T
        timestamp = packet[4]
        measurements.append(np.matrix([[packet[5]],
                                           [packet[6]],
                                           [packet[7]],
                                           [packet[8]]]))
        
    if not is_initialized:
        if packet[0] == 'L':
            Px, Py, vx, vy = packet[1], packet[2], 0, 0
        elif packet[0] == 'R' and not only_lidar:
            phi_measured = packet[1]
            rho_measured = packet[2]
            rhodot_measured = packet[3]
            Px, Py = (cos(phi_measured) * rho_measured, sin(phi_measured) * rho_measured)
            vx, vy = (rhodot_measured * cos(phi_measured) , rhodot_measured * sin(phi_measured))
        if abs(Px + Py) <= 0.0001:
            Px = 0.0001
            Py = 0.0001
        x = np.matrix([[Px,Py,vx,vy]]).T
        prev_timestamp = timestamp
        is_initialized = True
        estimations.append(x)
    else:
        dt = (timestamp - prev_timestamp) / 1000000.0
        prev_timestamp = timestamp
        F = computeFmatrix(dt)
        Q = computeQmatrix(dt, 9, 9)
        
        x = F * x
        P = (F * P * F.T) + Q
        
        if packet[0] == 'L':
            y = z - HL * x

            PHLt = P * HL.T
            S = HL * PHLt + RL
            K = PHLt * S.I

            x += K * y
            P = (xI - K * HL) * P
            estimations.append(x)
        elif packet[0] == 'R' and not only_lidar:
            Px,Py,vx,vy = x[0][0,0], x[1][0,0], x[2][0,0], x[3][0,0]
            ro = sqrt(Px**2 + Py**2)

            phi = atan2(Py,Px)
            ro_dot = (Px*vx + Py*vy) / ro
            
            y = z - np.matrix([ro, phi, ro_dot]).T
            
            while (y[1] > pi): y[1] -= 2.*pi
            while (y[1] < -pi): y[1] += 2.*pi
            
            HR = computeRadarJacobian(x)
            
            S = HR * P * HR.T + RR
            K = P * HR.T * S.I
            x += K * y


            P = (xI - K * HR) * P
            estimations.append(x)
        
rmse = computeRMSE(estimations, measurements).flatten()
print("RMSE = {}".format(rmse))

RMSE = [[0.12219136 0.09837984 0.58251275 0.45669849]]


In [394]:
#QUESTION NUMBER 4
#ANSWER IS RMSE = [[0.09722562 0.08537612 0.45085468 0.43958819]]
import pandas as pd
import numpy as np
from math import *

def computeRMSE(EstimateVector, trueVector):
    if len(EstimateVector) != len(trueVector) or len(EstimateVector) == 0:
        raise ValueError('computeRMSE() - Error - EstimateVector and trueVector not the same length.')

    rmse = np.matrix([0., 0., 0., 0.]).T
    for est, gt in zip(EstimateVector, trueVector):
        rmse += np.square(est - gt)

    rmse /= len(EstimateVector)
    return np.sqrt(rmse)

def computeRadarJacobian(Xvector):
    Px,Py,vx,vy = (Xvector[0][0,0],Xvector[1][0,0],Xvector[2][0,0],Xvector[3][0,0])
    
    px2_py2 = Px**2 + Py**2
    sqrt_px2_py2 = sqrt(px2_py2)
    px2_py2_sqrt_px2_py2 = (px2_py2 * sqrt_px2_py2)

    if px2_py2 < 0.0001:
        return np.matrix([[0, 0, 0, 0],
                         [0, 0, 0, 0],
                         [0, 0, 0, 0]])

    place11 = Px / sqrt_px2_py2
    place12 = Py / sqrt_px2_py2
    place21 = -Py / px2_py2
    place22 = Px / px2_py2
    place31 = Py * (vx * Py - vy * Px) / px2_py2_sqrt_px2_py2
    place32 = Px * (Px * vy - Py * vx) / px2_py2_sqrt_px2_py2

    return np.matrix([[place11, place12, 0, 0],
                      [place21, place22, 0, 0],
                      [place31, place32, place11, place12]])
def computeCovMatrix(deltaT, sigma_aX, sigma_aY):
    pass
def computeFmatrix(deltaT):
    mat = np.matrix([[1, 0, deltaT, 0],
           [0, 1, 0, deltaT],
           [0, 0, 1, 0],
           [0, 0, 0, 1]])
    return mat
def computeQmatrix(dt, noise_ax, noise_ay):
    delta2 = dt**2
    delta3 = dt**3
    delta4 = dt**4

    mat = np.matrix([[delta4 * noise_ax / 4, 0, delta3 * noise_ax / 2, 0],
                     [0, delta4 * noise_ay / 4, 0, delta3 * noise_ay / 2],
                     [delta3 * noise_ax / 2, 0, delta2 * noise_ax, 0],
                     [0, delta3 * noise_ay / 2, 0, delta2 * noise_ay]])
    return mat




data = pd.read_csv("obj_pose-laser-radar-synthetic-input.txt",                  
                header=None,                           
                sep='\t',                              
                names=['x'+str(x) for x in range(11)]) 
dt = 0.1
timestamp, prev_timestamp = 0, 0
Px, Py, vx, vy = 0, 0, 0, 0
x = None
P = np.matrix([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1000, 0], [0, 0, 0, 1000]])
HL = np.matrix([[1, 0, 0, 0], [0, 1, 0, 0]])
HR = np.matrix([[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]])
RL = np.matrix([[0.0225, 0], [0, 0.0225]])
RR = np.matrix([[0.09, 0, 0], [0, 0.0009, 0], [0, 0, 0.09]])
F = computeFmatrix(dt)
Q = None
xI = np.eye(4)
z  = 0
estimations = []   
measurements = []  

only_lidar = False
is_initialized = False
for _, packet in data.iterrows(): 
    if packet[0] == 'L':
        z = np.matrix([[packet[1], packet[2]]]).T
        timestamp = packet[3]
        measurements.append(np.matrix([[packet[4]],
                                           [packet[5]],
                                           [packet[6]],
                                           [packet[7]]]))
        
            
    elif packet[0] == 'R' and not only_lidar:
        z = np.matrix([[packet[1], packet[2], packet[3]]]).T
        timestamp = packet[4]
        measurements.append(np.matrix([[packet[5]],
                                           [packet[6]],
                                           [packet[7]],
                                           [packet[8]]]))
        
    if not is_initialized:
        if packet[0] == 'L':
            Px, Py, vx, vy = packet[1], packet[2], 0, 0
        elif packet[0] == 'R' and not only_lidar:
            phi_measured = packet[1]
            rho_measured = packet[2]
            rhodot_measured = packet[3]
            Px, Py = (cos(phi_measured) * rho_measured, sin(phi_measured) * rho_measured)
            vx, vy = (rhodot_measured * cos(phi_measured) , rhodot_measured * sin(phi_measured))
        if abs(Px + Py) <= 0.0001:
            Px = 0.0001
            Py = 0.0001
        x = np.matrix([[Px,Py,vx,vy]]).T
        prev_timestamp = timestamp
        is_initialized = True
        estimations.append(x)
    else:
        dt = (timestamp - prev_timestamp) / 1000000.0
        prev_timestamp = timestamp
        F = computeFmatrix(dt)
        Q = computeQmatrix(dt, 9, 9)
        
        x = F * x
        P = (F * P * F.T) + Q
        
        if packet[0] == 'L':
            y = z - HL * x

            PHLt = P * HL.T
            S = HL * PHLt + RL
            K = PHLt * S.I

            x += K * y
            P = (xI - K * HL) * P
            estimations.append(x)
        elif packet[0] == 'R' and not only_lidar:
            Px,Py,vx,vy = x[0][0,0], x[1][0,0], x[2][0,0], x[3][0,0]
            ro = sqrt(Px**2 + Py**2)

            phi = atan2(Py,Px)
            ro_dot = (Px*vx + Py*vy) / ro
            
            y = z - np.matrix([ro, phi, ro_dot]).T
            
            while (y[1] > pi): y[1] -= 2.*pi
            while (y[1] < -pi): y[1] += 2.*pi
            
            HR = computeRadarJacobian(x)
            
            S = HR * P * HR.T + RR
            K = P * HR.T * S.I
            x += K * y


            P = (xI - K * HR) * P
            estimations.append(x)
        
rmse = computeRMSE(estimations, measurements).flatten()
print("RMSE = {}".format(rmse))

RMSE = [[0.09722562 0.08537612 0.45085468 0.43958819]]
